In [1]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd

In [2]:
earliest_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT initialissuedate FROM business_licenses order by initialissuedate limit 1
"""
).json()["rows"][0]["initialissuedate"]

latest_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT mostrecentissuedate FROM business_licenses order by mostrecentissuedate desc limit 1
"""
).json()["rows"][0]["mostrecentissuedate"]

n_licenses = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM business_licenses
"""
).json()["rows"][0]["count"]
print(
    f"There are {n_licenses} licenses between {earliest_license} and {latest_license}"
)

There are 353245 licenses between 1993-06-01T00:00:00Z and 3200-12-31T00:00:00Z


In [3]:
license_statuses = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT distinct(licensestatus) FROM business_licenses
"""
).json()["rows"]
print(license_statuses)

[{'licensestatus': 'Suspended'}, {'licensestatus': 'Active'}, {'licensestatus': 'Closed'}, {'licensestatus': 'Expired'}, {'licensestatus': 'Revoked'}, {'licensestatus': 'Inactive'}]


In [43]:
license_statuses_df = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT * FROM business_licenses bl, opa_properties_public opa
where bl.opa_account_num = opa.parcel_number
and licensetype='Rental' and initialissuedate>'2005-01-01'
AND opa.zoning like 'R%'
limit 10000
"""
).json()["rows"]
df_test = pd.DataFrame(license_statuses_df)[['opa_account_num','legalname','legalentitytype','business_name','opa_owner', 'mostrecentissuedate']]

In [48]:
df_test.sample(10)

,opa_account_num,legalname,legalentitytype,business_name,opa_owner,mostrecentissuedate
1330,662615791,ALAM SHAHJABIN,Individual,KELLY DECESARE REAL PROP MGNT,ALAM SHAHJABIN,2015-07-22T00:00:00Z
2022,404216850,M & V REAL ESTATE MGMNT INC,Company,M & V REAL ESTATE MGMNT INC,"JOSEPH MANGO, VACARINO NICHOLAS",2021-03-03T00:00:00Z
9592,513273500,WORKFORCE HOMES 3 LP,Company,WORKFORCE HOMES 3 LP,WORKFORCE HOMES 3 LP,2019-12-20T00:00:00Z
1616,243234200,DORMEUS KETLEEN,Individual,DORMEUS KETLEEN,DORMEUS KATLEEN,2015-05-08T00:00:00Z
471,881299030,RENAISSANCE COMMUNITY DEV CORP,Company,RENAISSANCE COMMUNITY DEV CORP,RENAISSANCE GARDEN APARTM,2017-12-15T00:00:00Z
1848,351363900,JIAN CHAI,Individual,JIAN CHAI,"LIN JIAN CHAI, ZHANG JIN XIU",2016-07-24T00:00:00Z
565,341107300,DAMSELFLY LAND HOLDING LLC,Company,DAMSELFLY LAND HOLDING LLC,DUKE STACEY,2020-03-03T00:00:00Z
1065,411390000,LEMAIRE JEAN,Individual,LEMAIRE JEAN,LEMAIRE JEAN,2012-03-03T10:46:43Z
4092,332520800,KAUFF HOWARD,Individual,HOWARD KAY KAUFF,KAUFF HOWARD KAY,2021-01-29T00:00:00Z
1925,886612705,"PARTNERSHIP PRESERVATION ASSOCIATES, L.P.",Company,"PARTNERSHIP PRESERVATION ASSOCIATES, L.P.",PARTNERSHIP PRESERVATION,2020-01-31T00:00:00Z


In [6]:
license_statuses = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT distinct(legalentitytype) FROM business_licenses
"""
).json()["rows"]
print(license_statuses)

[{'legalentitytype': None}, {'legalentitytype': 'Individual'}, {'legalentitytype': 'Company'}]


In [7]:
n_rental_licenses = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM business_licenses where licensetype='Rental'
"""
).json()["rows"][0]["count"]


earliest_rental_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT initialissuedate FROM business_licenses where licensetype='Rental' order by initialissuedate limit 1
"""
).json()["rows"][0]["initialissuedate"]

latest_rental_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT mostrecentissuedate FROM business_licenses where licensetype='Rental' order by mostrecentissuedate desc limit 1
"""
).json()["rows"][0]["mostrecentissuedate"]

print(
    f"There are {n_rental_licenses} rental licenses ({n_rental_licenses/n_licenses*100:.2f}% of total) between {earliest_rental_license} and {latest_rental_license}"
)

There are 234096 rental licenses (66.38% of total) between 1993-06-01T00:00:00Z and 2021-03-03T21:08:03Z


In [8]:
n_active_rental_licenses = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM business_licenses where licensetype='Rental' and licensestatus='Active'
"""
).json()["rows"][0]["count"]


earliest_active_rental_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT mostrecentissuedate FROM business_licenses where licensetype='Rental' and licensestatus='Active' order by initialissuedate limit 1
"""
).json()["rows"][0]["mostrecentissuedate"]

latest_active_rental_license = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT mostrecentissuedate FROM business_licenses where licensetype='Rental' and licensestatus='Active' order by mostrecentissuedate desc limit 1
"""
).json()["rows"][0]["mostrecentissuedate"]

print(
    f"There are {n_active_rental_licenses} active rental licenses"
    f" ({n_active_rental_licenses/n_licenses*100:.2f}% of total) between {earliest_active_rental_license} and {latest_active_rental_license}"
)

There are 60865 active rental licenses (17.26% of total) between 2021-01-22T00:00:00Z and 2021-03-03T21:08:03Z


In [9]:
n_matching_addresses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) 
    FROM business_licenses bl, opa_properties_public opp 
    where bl.licensetype='Rental' and bl.opa_account_num= opp.parcel_number
"""
).json()["rows"][0]["count"]
print(
    f"There are {n_matching_addresses} rental licenses that match a property in the properties dataset."
)

n_matching_addresses_active = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) 
    FROM business_licenses bl, opa_properties_public opp 
    where bl.licensetype='Rental' and bl.licensestatus = 'Active' and bl.opa_account_num= opp.parcel_number
"""
).json()["rows"][0]["count"]
print(
    f"There are {n_matching_addresses_active} active rental licenses that match a property in the properties dataset."
)

There are 159258 rental licenses that match a property in the properties dataset.
There are 58905 active rental licenses that match a property in the properties dataset.


In [10]:
n_matching_addresses_exempt = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) FROM business_licenses bl, opa_properties_public opp where bl.licensetype='Rental' and
    bl.opa_account_num= opp.parcel_number and (opp.homestead_exemption > 0 and opp.homestead_exemption is not null)
    
"""
).json()["rows"][0]["count"]
print(
    f"There are {n_matching_addresses_exempt} rental licenses"
    " that match a property with a homestead exemption in the properties dataset."
)

There are 20347 rental licenses that match a property with a homestead exemption in the properties dataset.


In [11]:
matching_addresses_exempt = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT opp.parcel_number, bl.business_mailing_address, bl.business_name, bl.legalname, 
    bl.licensestatus, opp.homestead_exemption, bl.mostrecentissuedate, bl.expirationdate,
    opp.location, opp.unit, opp.mailing_address_1, opp.mailing_street, opp.mailing_city_state 
    FROM business_licenses bl, opa_properties_public opp where bl.licensetype='Rental' and
    bl.opa_account_num= opp.parcel_number and (opp.homestead_exemption > 0 and opp.homestead_exemption is not null)
    and bl.licensestatus = 'Active'
"""
).json()["rows"]

match_address_exempt_df = pd.DataFrame(matching_addresses_exempt)
print(
    f"There are {len(match_address_exempt_df)} properties with both homestead exempetions and rental licenses."
)
match_address_exempt_df.head()

There are 3784 properties with both homestead exempetions and rental licenses.


,parcel_number,business_mailing_address,business_name,legalname,licensestatus,homestead_exemption,mostrecentissuedate,expirationdate,location,unit,mailing_address_1,mailing_street,mailing_city_state
0,652284000,"8206 DITMAN ST 1ST FLOOR PHILADELPHIA, PA 191...",THOMAS CAROLYN,THOMAS CAROLYN,Active,20000,2021-02-11T00:00:00Z,2022-02-28T00:00:00Z,8206 DITMAN ST,None,None,None,None
1,406655902,"7303 SANDERLING PL 2ND FL PHILADELPHIA, PA 19...",BEKELE FIKERTE,BEKELE FIKERTE,Active,45000,2021-02-04T00:00:00Z,2022-02-28T00:00:00Z,7303 SANDERLING PL,None,2ND FLOOR,7303 SANDERLING PL,PHILADELPHIA PA
2,022153300,"2228 HILLTOP VIEW RD UNIONVILLE, PA 19375 USA",PHILLYPROP L P,PHILLYPROP L P,Active,45000,2021-02-25T00:00:00Z,2022-02-28T00:00:00Z,805 S 5TH ST,None,None,None,None
3,871042550,"812 04TH ST PHILADELPHIA, PA 19147 USA",MICHAEL AND MARIE,FAHERTY MARIE,Active,20000,2021-02-14T00:00:00Z,2022-02-28T00:00:00Z,812 S 4TH ST,None,None,318 FEDERAL ST,PHILADELPHIA PA
4,481142700,"1529 W PASSYUNK AVENUE PHILADELPHIA, PA 19145...",STRUNK ADELINE,STRUNK ADELINE,Active,45000,2021-02-23T00:00:00Z,2022-02-28T00:00:00Z,1529 W PASSYUNK AVE,None,None,None,None


In [12]:
requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT * FROM business_licenses where licensetype='Rental' limit 1
"""
).json()["rows"][0]

{'cartodb_id': 1,
 'the_geom': None,
 'the_geom_webmercator': None,
 'objectid': 1,
 'addressobjectid': 15698520,
 'address': None,
 'unit_type': None,
 'unit_num': None,
 'zip': None,
 'censustract': None,
 'parcel_id_num': None,
 'opa_account_num': None,
 'opa_owner': None,
 'licensenum': '000001',
 'revenuecode': '3202',
 'licensetype': 'Rental',
 'initialissuedate': '2004-05-17T00:00:00Z',
 'mostrecentissuedate': '2020-02-18T00:00:00Z',
 'expirationdate': '2021-02-28T00:00:00Z',
 'inactivedate': None,
 'licensestatus': 'Expired',
 'numberofunits': 44,
 'owneroccupied': 'No',
 'legalfirstname': None,
 'legallastname': None,
 'legalname': '509 VINE STREET TCE LP',
 'legalentitytype': 'Company',
 'business_name': '509 VINE STREET TCE LP',
 'business_mailing_address': '4328 Ridge Ave PHILADELPHIA, PA  19129 USA',
 'geocode_x': None,
 'geocode_y': None}

In [14]:
nonmatching_licenses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT address, legalname, business_name, business_mailing_address, opa_account_num
    FROM business_licenses bl where bl.opa_account_num not in (select parcel_number from opa_properties_public) and bl.licensetype='Rental' and bl.licensestatus='Active'
    
"""
).json()["rows"]
nonmatching_licenses_df = pd.DataFrame(nonmatching_licenses)

print(
    f"It appears that there exists {len(nonmatching_licenses)} error(s) in the data, but otherwise active licenses can be traced to properties."
)
nonmatching_licenses_df

It appears that there exists 1 error(s) in the data, but otherwise active licenses can be traced to properties.


,address,legalname,business_name,business_mailing_address,opa_account_num
0,1005 N MARSHALL ST,1005 MARSHALL ST LLC,1005 MARSHALL ST LLC,"2600 Route 130 Cinnaminson, NJ 08077 USA",871084350


In [15]:
matching_licenses_result = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT bl.legalfirstname, bl.legallastname, bl.legalname, bl.opa_owner, bl.legalentitytype, bl.business_name, bl.business_mailing_address,
    opp.parcel_number, opp.mailing_address_1, opp.mailing_address_2, opp.mailing_street, opp.mailing_city_state, opp.owner_1, opp.owner_2, opp.location
    FROM business_licenses bl, opa_properties_public opp where bl.opa_account_num= opp.parcel_number and bl.licensetype='Rental' and bl.licensestatus='Active' limit 1000
    
"""
).json()["rows"]

In [16]:
pd.DataFrame(matching_licenses_result).head()

,legalfirstname,legallastname,legalname,opa_owner,legalentitytype,business_name,business_mailing_address,parcel_number,mailing_address_1,mailing_address_2,mailing_street,mailing_city_state,owner_1,owner_2,location
0,None,None,HIN RATHA,APSARA LLC,Company,HIN RATHA,"2 LONG BOW DRIVE SEWELL, NJ 08080 USA",871554970,None,None,None,None,APSARA LLC,None,700 RITNER ST
1,None,None,NUTRUST,NU TRUST,Company,NUTRUST,"4715 05TH ST PHILADELPHIA, PA 19120 USA",453165500,None,None,4715 N 5TH ST,PHILADELPHIA PA,NU TRUST,None,3921 ARCADIA ST
2,None,None,NUTRUST,NU TRUST,Company,NUTRUST,"4715 05TH ST PHILADELPHIA, PA 19120 USA",071461600,None,None,4715 N 5TH ST,PHILADELPHIA PA,NU TRUST,None,3109 ARBOR ST
3,None,None,NUTRUST,NU TRUST,Company,NUTRUST,"4715 05TH ST PHILADELPHIA, PA 19120 USA",422015400,None,None,4715 N 5TH ST,PHILADELPHIA PA,NU TRUST,None,423 W ANNSBURY ST
4,None,None,ZETA TRUST,ZETA TRUST,Company,ZETA TRUST,"4715 05TH ST PHILADELPHIA, PA 19120 USA",073121700,None,None,None,None,ZETA TRUST,None,3459 ORMES ST


In [17]:
n_matching_addresses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) FROM business_licenses bl, opa_properties_public opp where bl.opa_account_num= opp.parcel_number
    and opp.location = bl.address
"""
).json()["rows"][0]["count"]

n_non_matching_addresses = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=SELECT count(distinct(opp.parcel_number)) FROM business_licenses bl, opa_properties_public opp where bl.opa_account_num= opp.parcel_number
    and opp.location != bl.address
"""
).json()["rows"][0]["count"]

pct = (
    round(
        n_non_matching_addresses / (n_non_matching_addresses + n_matching_addresses), 2
    )
    * 100
)
print(
    f"There are {n_matching_addresses} properties that perfectly match the opa_properties_public.location to business_licenses.address"
)
print(
    f"There are {n_non_matching_addresses} properties that do not perfectly match the opa_properties_public.location to business_licenses.address, about {pct}%"
)
print("However, a spot check seems like they are in fact the same property.")

There are 178754 properties that perfectly match the opa_properties_public.location to business_licenses.address
There are 9934 properties that do not perfectly match the opa_properties_public.location to business_licenses.address, about 5.0%
However, a spot check seems like they are in fact the same property.


### Other notes about this data

In [18]:
license_type_results = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT distinct(licensetype) FROM business_licenses
"""
).json()["rows"]
license_types = [l["licensetype"] for l in license_type_results]
license_types_str = "\n".join(sorted(license_types))
print(f"List of License Types (licensetype):\n{license_types_str}")

List of License Types (licensetype):
Amusement
Auto Wrecking / Junk Yard
Bingo
Carnival
Child Care Facility
Curb Market
Dumpster License - Construction
Dumpster License - Private Property
Dumpster License - Public ROW
Food Caterer
Food Estab, Retail Non-Permanent Location (Annual)
Food Estab, Retail Non-Permanent Location (Event)
Food Establishment, Outdoor
Food Establishment, Retail Perm Location (Large)
Food Establishment, Retail Permanent Location
Food Manufacturer / Wholesaler
Food Preparing and Serving
Food Preparing and Serving (30+ SEATS)
Handbill Distribution
Hazardous Materials
High Rise
Honor Box
Hot Work
Limited Occasion
Motor Vehicle Repair / Retail Mobile Dispensing
Outdoor Advertising Sign
Pawn Shop
Precious Metal Dealer
Promoter Registration
Public Garage / Parking Lot
Rental
Residential Property Wholesaler
Scales and Scanners
Sidewalk Cafe
Sidewalk Cafe/Streetery (Temporary)
Small Games Of Chance
Special Assembly Occupancy
Special Permit
Tow Company
Towing
Vacant Commer